<a href="https://colab.research.google.com/github/yanliu1111/python-tutorial/blob/main/Customize_RAG_with_LlamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.5/846.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


# Setup

#### Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2023-09-29 04:49:09--  https://raw.githubusercontent.com/jerryjliu/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75047 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.29K  --.-KB/s    in 0.008s  

2023-09-29 04:49:09 (9.48 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75047/75047]



#### Load Data

In [ ]:
from llama_index import SimpleDirectoryReader

reader = SimpleDirectoryReader("./data/paul_graham/")
documents = reader.load_data()

/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing BasePromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(


# Building QA System with OpenSource LLM

In [ ]:
from llama_index.llms.anyscale import Anyscale
from llama_index import ServiceContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
import openai

ANYSCALE_ENDPOINT_TOKEN = "esecret_lail63z8bcg3vgnaehersycjkp"
openai.api_key = 'YOUR OPENAI API KEY'

# Define LLM
llm = Anyscale(model = "meta-llama/Llama-2-13b-chat-hf",
                 api_key=ANYSCALE_ENDPOINT_TOKEN)

# Define Embedding Model
embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

# Abstract llm, embedding model
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
)

# Create index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Setup Query Engine
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("why did paul graham start YC?")

In [ ]:
from IPython.display import display, HTML
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

# Building QA System with OpenSource LLM and Embeddings.

In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding

# Define LLM
llm = Anyscale(model = "meta-llama/Llama-2-13b-chat-hf",
                 api_key=ANYSCALE_ENDPOINT_TOKEN)

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Abstract llm, embedding model
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
)

# Create index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
# Setup Query Engine
query_engine = index.as_query_engine()

In [ ]:
%%timeit -r 1 -n 1
response = query_engine.query("why did paul graham start YC?")

from IPython.display import display, HTML
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

4.08 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Let's use Optimum Embeddings from HuggingFace

You can install the dependencies with `pip install transformers optimum[exporters]`.


In [ ]:
!pip install transformers optimum[exporters]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.

First, we need to create the ONNX model. ONNX models provide improved inference speeds, and can be used across platforms (i.e. in TransformersJS)

In [ ]:
from llama_index.embeddings import OptimumEmbedding

OptimumEmbedding.create_and_save_optimum_model("BAAI/bge-small-en-v1.5", "./bge_onnx")

Framework not specified. Using pt to export to ONNX.
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Saved optimum model to ./bge_onnx. Use it with `embed_model = OptimumEmbedding(folder_name='./bge_onnx')`.


In [ ]:
# load the embedding model
embed_model = OptimumEmbedding(folder_name="./bge_onnx")

In [ ]:
# Define LLM
llm = Anyscale(model = "meta-llama/Llama-2-13b-chat-hf",
                 api_key=ANYSCALE_ENDPOINT_TOKEN)

# Abstract llm, embedding model
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
)

# Create index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
# Create Query Engine
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("why did paul graham start YC?")

from IPython.display import display, HTML
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

9.35 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Customizing the chunk size, chunk overlap and LLM context window, number of output tokens.

In [ ]:
from llama_index import ServiceContext, LLMPredictor, PromptHelper, VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.node_parser import SimpleNodeParser
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms.anyscale import Anyscale

# Define LLM
ANYSCALE_ENDPOINT_TOKEN = "esecret_lail63z8bcg3vgnaehersycjkp"
llm = Anyscale(model = "meta-llama/Llama-2-13b-chat-hf",
                 api_key=ANYSCALE_ENDPOINT_TOKEN)

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Create Node Parser
node_parser = SimpleNodeParser.from_defaults(chunk_size=2000, chunk_overlap=100)

# Create PromptHelper
prompt_helper = PromptHelper(
  context_window=4096,
  num_output=512,
  chunk_overlap_ratio=0.1,
)

# Customise LLM, Embedding model, Node parser and Prompthelper
service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
  node_parser=node_parser,
  prompt_helper=prompt_helper
)

# Create Index
index = VectorStoreIndex.from_documents(documents, service_context = service_context)

In [ ]:
# Setup Query Engine
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("why did paul graham start YC?")

from IPython.display import display, HTML
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

# Saving and Loading the Index

In [ ]:
from llama_index import StorageContext, load_index_from_storage
from llama_index.node_parser import SimpleNodeParser

# create parser and parse document into nodes
node_parser = SimpleNodeParser.from_defaults(chunk_size=2000, chunk_overlap=100)
nodes = node_parser.get_nodes_from_documents(documents)

# create storage context using default stores
storage_context = StorageContext.from_defaults()

# # build index
index = VectorStoreIndex(nodes, storage_context=storage_context, service_context = service_context)

# save index
index.storage_context.persist(persist_dir="storage")

In [ ]:
# to load index later, make sure you setup the storage context
# this will loaded the persisted stores from persist_dir
storage_context = StorageContext.from_defaults(persist_dir="storage")

# then load the index object
# if loading multiple indexes from a persist dir
loaded_index = load_index_from_storage(storage_context = storage_context, service_context=service_context)

# setup query engine
query_engine = loaded_index.as_query_engine(similarity_top_k=3)
response = query_engine.query("why did paul graham start YC?")

# print the synthesized response.
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

# Count Prompt Tokens and Checking underlying Prompt

In [ ]:
from llama_index import set_global_service_context
from llama_index.callbacks import CallbackManager, TokenCountingHandler
import tiktoken

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

llm = OpenAI(model='gpt-3.5-turbo')

service_context = ServiceContext.from_defaults(
    llm=llm, callback_manager=callback_manager
)

# set the global default!
set_global_service_context(service_context)

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
print(token_counter.total_embedding_token_count)

16662


Let's reset embedding count.



In [ ]:
token_counter.reset_counts()

In [ ]:
print(token_counter.total_embedding_token_count)

0


In [ ]:
query_engine = index.as_query_engine(similarity_top_k=4)
response = query_engine.query("Why did author start YC?")

In [ ]:
print(
    "Embedding Tokens: ",
    token_counter.total_embedding_token_count,
    "\n",
    "LLM Prompt Tokens: ",
    token_counter.prompt_llm_token_count,
    "\n",
    "LLM Completion Tokens: ",
    token_counter.completion_llm_token_count,
    "\n",
    "Total LLM Token Count: ",
    token_counter.total_llm_token_count,
    "\n",
)

Embedding Tokens:  15 
 LLM Prompt Tokens:  9037 
 LLM Completion Tokens:  310 
 Total LLM Token Count:  9347 



In [ ]:
print("prompt: ", token_counter.llm_token_counts[0].prompt, "...\n")
print(
    "prompt token count: ", token_counter.llm_token_counts[0].prompt_token_count, "\n"
)

print("completion: ", token_counter.llm_token_counts[0].completion, "...\n")
print(
    "completion token count: ",
    token_counter.llm_token_counts[0].completion_token_count,
    "\n",
)

print("total token count", token_counter.llm_token_counts[0].total_token_count)

prompt:  system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 